In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import pyspark.pandas as ps

/home4/tc737978/Documents/M2/Machine Learning/TD/TD1/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Initialisation de Spark

In [2]:
spark = SparkSession.builder.appName("Projet InfoD").config("spark.jars", "/home1/tc737978/Documents/M2/Info décisionnelle/postgresql-42.7.5.jar").config("spark.driver.memory", "10g").config("spark.executor.memory", "10g").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)


25/02/21 12:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Chargement des données

In [3]:
tip = spark.read.csv("yelp_academic_dataset_tip.csv", header=True, inferSchema=True)
business_source = pd.read_json("yelp_academic_dataset_business.json", lines=True)
business_source = spark.createDataFrame(business_source)
checkin = spark.read.json("yelp_academic_dataset_checkin.json")
reviews = spark.read.jdbc("jdbc:postgresql://stendhal:5432/tpid2020", "yelp.review", properties={"user": "tpid", "password": "tpid", "driver": "org.postgresql.Driver"})

In [4]:
tip.show()

+--------------------+----------------+-------------------+--------------------+--------------------+
|         business_id|compliment_count|               date|                text|             user_id|
+--------------------+----------------+-------------------+--------------------+--------------------+
|UYX5zL_Xj9WEc_Wp-...|               0|2013-11-26 18:20:08|Here for a quick mtg|hf27xTME3EiCp6NL6...|
|Ch3HkwQYv1YKw_FO0...|               0|2014-06-15 22:26:45|Cucumber strawber...|uEvusDwoSymbJJ0au...|
|rDoT-MgxGRiYqCmi0...|               0|2016-07-18 22:03:42|Very nice good se...|AY-laIws3S7YXNl_f...|
|OHXnDV01gLokiX1EL...|               0|2014-06-06 01:10:34|It's a small plac...|Ue_7yUlkEbX4AhnYd...|
|GMrwDXRlAZU2zj5nH...|               0|2011-04-08 18:12:01|8 sandwiches, $24...|LltbT_fUMqZ-ZJP-v...|
|ALwAlxItASeEs2vYA...|               0|2015-05-20 20:17:38|Great ramen! Not ...|HHNBqfbDR8b1iq-QG...|
|d_L-rfS1vT3JMzgCU...|               0|2014-09-01 01:23:48|Cochinita Pibil w...|r0

In [ ]:
business_source.show()

25/02/20 16:08:11 WARN TaskSetManager: Stage 3 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+---------------+-----+-----------+-------------+---------------+-----+------------+-------+--------------------+--------------------+--------------------+
|         business_id|                name|             address|           city|state|postal_code|     latitude|      longitude|stars|review_count|is_open|          attributes|          categories|               hours|
+--------------------+--------------------+--------------------+---------------+-----+-----------+-------------+---------------+-----+------------+-------+--------------------+--------------------+--------------------+
|f9NumwFMBDn751xgF...|The Range At Lake...|     10913 Bailey Rd|      Cornelius|   NC|      28031|   35.4627242|    -80.8526119|  3.5|          36|      1|{BikeParking -> T...|Active Life, Gun/...|{Thursday -> 11:0...|
|Yzvjg0SayhoZgCljU...|   Carlos Santo, NMD|8880 E Via Linda,...|     Scottsdale|   AZ|      85258|   33.5694041|   -111.8902

In [5]:
checkin.show()

+--------------------+--------------------+
|         business_id|                date|
+--------------------+--------------------+
|--1UhMGODdWsrMast...|2016-04-26 19:49:...|
|--6MefnULPED_I942...|2011-06-04 18:22:...|
|--7zmmkVg-IMGaXbu...|2014-12-29 19:25:...|
|--8LPVSo5i0Oo61X0...| 2016-07-08 16:43:30|
|--9QQLMTbFzLJ_oT-...|2010-06-26 17:39:...|
|--9e1ONYQuAa-CB_R...|2010-02-08 05:56:...|
|--DaPTJW3-tB1vP-P...|2012-06-03 17:46:...|
|--DdmeR16TRb3LsjG...|2012-11-02 21:26:...|
|--EF5N7P70J_UYBTP...|2018-05-25 19:52:...|
|--EX4rRznJrltyn-3...|2010-02-26 17:05:...|
|--FBCX-N37CMYDfs7...|2010-05-31 07:57:...|
|--FLdgM0GNpXVMn74...|2012-10-23 18:47:...|
|--FnvijzY20d1nk9H...|2019-09-19 02:42:...|
|--GM_ORV2cYS-h38D...|2011-09-11 18:16:...|
|--I7YYLada0tSLkOR...|2014-11-07 00:51:...|
|--KCl2FvVQpvjzmZS...|2011-07-29 16:53:...|
|--KQsXc-clkO7oHRq...|2010-05-02 23:57:...|
|--Ni3oJ4VOqfOEu7S...| 2019-06-07 17:54:58|
|--Rsj71PBe31h5Ylj...|2011-12-15 18:09:...|
|--S62v0QgkqQaVUhF...|2010-12-25

In [4]:
reviews.show()

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+---------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|spark_partition|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+---------------+
|INzLc1Y0JNQzIHjPH...|   1|2016-10-03|    0|1XeoWasllr6kxDHXx...|  2.0|It's average. Not...|     0|7A6lEhTsGghPFMvbz...|             45|
|-FLnsWAa4AGEW4NgE...|   0|2017-05-03|    0|9LXbXfMj1JxtTHac4...|  5.0|Loved this place!...|     0|VzAS-njv-GKcUnA7D...|             46|
|7tcfLr-QpzJF-y5df...|   0|2014-11-17|    0|rdr2PiMFV58PnmuKq...|  5.0|Delectably delici...|     1|JWsWxfaJs_VlHIyLQ...|             47|
|Ve6bA_jf9A7EWKXey...|   0|2015-12-07|    0|OrF8fvoQj60UPMITw...|  5.0|This Pizza shop h...|     0|kxpqvSYwq8JzRslmO...|             48|
|3WRvQJS9GIBubGxbH...|   0|2015-07-05|   

## Création des dataframes

### Business

In [13]:
Business = business_source.select("business_id")
Business = Business.withColumn("ID_Categorie", monotonically_increasing_id())
Business = Business.withColumn("ID_Attributes", monotonically_increasing_id())
Business = Business.withColumn("ID_Hours", monotonically_increasing_id())
Business = Business.withColumn("ID_Localisation", monotonically_increasing_id())
Business = Business.withColumn("ID_Reviews", monotonically_increasing_id())
Stars = reviews.select("business_id", "stars").groupBy("business_id").agg(avg("stars").alias("avg_stars")).na.fill(0)
Business = Business.join(Stars, on="business_id", how="left")
Business.show()


25/02/21 12:45:04 WARN TaskSetManager: Stage 123 contains a task of very large size (5746 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------+-------------+------------+---------------+------------+------------------+
|         business_id|ID_Categorie|ID_Attributes|    ID_Hours|ID_Localisation|  ID_Reviews|         avg_stars|
+--------------------+------------+-------------+------------+---------------+------------+------------------+
|-EnsDiGk4UhHQx-E4...|120259084294| 120259084294|120259084294|   120259084294|120259084294| 2.230769230769231|
|-yXkMzPPQ2PvzIC-N...|111669149700| 111669149700|111669149700|   111669149700|111669149700|1.3636363636363635|
|02LPDp1m9V8EcS79d...|128849018885| 128849018885|128849018885|   128849018885|128849018885|               4.0|
|0nc1B-3rKnvFawMtf...| 77309411341|  77309411341| 77309411341|    77309411341| 77309411341|1.5454545454545454|
|2ifCqFUJDgGPjdeY7...|103079215104| 103079215104|103079215104|   103079215104|103079215104|1.6666666666666667|
|3LF3HqwhUJYT9ma48...| 85899345929|  85899345929| 85899345929|    85899345929| 85899345929| 4.470588235294118|
|

### Localisation

In [17]:
Localisation = Business.select("ID_Localisation", "business_id")
Localisation = Localisation.join(business_source.select("business_id", "state", "city", "postal_code", "address", "latitude", "longitude"), on="business_id", how="left")
Localisation = Localisation.withColumn("country", \
                        when(Localisation["state"] == "NC", "USA") \
                        .when(Localisation["state"] == "AZ", "USA") \
                        .when(Localisation["state"] == "QC", "CAN") \
                        .when(Localisation["state"] == "NV", "USA") \
                        .when(Localisation["state"] == "IL", "USA") \
                        .when(Localisation["state"] == "AB", "CAN") \
                        .when(Localisation["state"] == "PA", "USA") \
                        .when(Localisation["state"] == "WI", "USA") \
                        .when(Localisation["state"] == "SC", "USA") \
                        .when(Localisation["state"] == "OH", "USA") \
                        .when(Localisation["state"] == "CA", "USA") \
                        .when(Localisation["state"] == "TX", "USA") \
                        .when(Localisation["state"] == "NY", "USA") \
                        .when(Localisation["state"] == "CO", "USA") \
                        .when(Localisation["state"] == "XWY", "USA") \
                        .when(Localisation["state"] == "GA", "USA") \
                        .when(Localisation["state"] == "BC", "CAN") \
                        .when(Localisation["state"] == "YT", "CAN") \
                        .when(Localisation["state"] == "HPL", "USA") \
                        .when(Localisation["state"] == "AL", "USA") \
                        .when(Localisation["state"] == "UT", "USA") \
                        .when(Localisation["state"] == "VT", "USA") \
                        .when(Localisation["state"] == "WA", "USA") \
                        .when(Localisation["state"] == "NE", "USA") \
                        .when(Localisation["state"] == "DOW", "USA") \
                        .when(Localisation["state"] == "MI", "USA") \
                        .when(Localisation["state"] == "FL", "USA") \
                        .when(Localisation["state"] == "AR", "USA") \
                        .when(Localisation["state"] == "HI", "USA") \
                        .when(Localisation["state"] == "MB", "CAN") \
                        .when(Localisation["state"] == "OR", "USA") \
                        .when(Localisation["state"] == "AK", "USA") \
                        .when(Localisation["state"] == "VA", "USA") \
                        .when(Localisation["state"] == "CT", "USA") \
                        .when(Localisation["state"] == "MO", "USA") \
                        .when(Localisation["state"] == "DUR", "USA") \
                        .when(Localisation["state"] == "ON", "CAN") \
                        .otherwise("Unknown"))
Localisation = Localisation.drop("business_id")
Localisation.show()

25/02/21 12:55:27 WARN TaskSetManager: Stage 230 contains a task of very large size (5746 KiB). The maximum recommended task size is 1000 KiB.
25/02/21 12:55:27 WARN TaskSetManager: Stage 231 contains a task of very large size (5746 KiB). The maximum recommended task size is 1000 KiB.


+---------------+-----+------------+-----------+--------------------+------------------+---------------+-------+
|ID_Localisation|state|        city|postal_code|             address|          latitude|      longitude|country|
+---------------+-----+------------+-----------+--------------------+------------------+---------------+-------+
|   120259084294|   AZ|    Chandler|      85225|  2060 N Arizona Ave|        33.3363385|   -111.8425591|    USA|
|   103079215116|   SC|   Fort Mill|      29732|  9775 Charlotte Hwy|         35.002024|     -80.856091|    USA|
|   111669149702|   OH|Strongsville|      44136|       9200 Pearl Rd|        41.3423081|       -81.8282|    USA|
|    17179869191|   AZ|  Scottsdale|      85251| 7800 E Camelback Rd|        33.5027727|   -111.9127297|    USA|
|   146028888066|   NV|   Las Vegas|      89109|  3068 S Highland Dr|        36.1342783|   -115.1743721|    USA|
|    34359738370|   NC|   Charlotte|      28216|6148-A Brookshire...|         35.287571|     -80

### Reviews

In [ ]:
nb_reviews = reviews.groupBy("business_id").count().withColumnRenamed("count", "Nb_Reviews")
nb_tips = tip.groupBy("business_id").count().withColumnRenamed("count", "Nb_Tips")
nb_checkins = checkin.withColumn("checkin", explode(split(col("date"), ", "))).groupBy("business_id").count().withColumnRenamed("count", "Nb_Checkins")
Reviews = Business.select("ID_Reviews", "business_id")
Reviews = Reviews.join(nb_reviews, Reviews.business_id == nb_reviews.business_id, "left").drop(nb_reviews.business_id)
Reviews = Reviews.join(nb_tips, Reviews.business_id == nb_tips.business_id, "left").drop(nb_tips.business_id)
Reviews = Reviews.join(nb_checkins, Reviews.business_id == nb_checkins.business_id, "left").drop(nb_checkins.business_id)
Reviews = Reviews.drop("business_id")
Reviews.show()

25/02/21 12:35:34 WARN TaskSetManager: Stage 21 contains a task of very large size (5746 KiB). The maximum recommended task size is 1000 KiB.


[Row(ID_Reviews=17179869191, Nb_Reviews=7, Nb_Tips=2, Nb_Checkins=340),
 Row(ID_Reviews=34359738370, Nb_Reviews=13, Nb_Tips=None, Nb_Checkins=5),
 Row(ID_Reviews=60129542144, Nb_Reviews=4, Nb_Tips=1, Nb_Checkins=8),
 Row(ID_Reviews=8589934598, Nb_Reviews=3, Nb_Tips=None, Nb_Checkins=14),
 Row(ID_Reviews=51539607561, Nb_Reviews=44, Nb_Tips=3, Nb_Checkins=30),
 Row(ID_Reviews=8589934596, Nb_Reviews=7, Nb_Tips=None, Nb_Checkins=8),
 Row(ID_Reviews=8589934600, Nb_Reviews=3, Nb_Tips=None, Nb_Checkins=None),
 Row(ID_Reviews=34359738374, Nb_Reviews=13, Nb_Tips=4, Nb_Checkins=181),
 Row(ID_Reviews=25769803779, Nb_Reviews=136, Nb_Tips=19, Nb_Checkins=176),
 Row(ID_Reviews=17179869190, Nb_Reviews=8, Nb_Tips=None, Nb_Checkins=2)]

### Year

### Categories

In [6]:
Categories = Business.select("ID_Categorie")
Categories.show()


+------------+
|ID_Categorie|
+------------+
|           0|
|           1|
|           2|
|           3|
|           4|
|           5|
|           6|
|           7|
|           8|
|           9|
|          10|
|          11|
|          12|
|          13|
|          14|
|          15|
|          16|
|          17|
|          18|
|          19|
+------------+
only showing top 20 rows



25/02/20 16:08:36 WARN TaskSetManager: Stage 5 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Categories_List

In [7]:
# Sélectionner les colonnes nécessaires
categories_df = business_source.select("business_id", "categories")

# Transformer la colonne categories en liste
categories_df = categories_df.withColumn("categories_list", split(categories_df["categories"], ", "))

# Exploser la liste des catégories pour obtenir une ligne par catégorie
categories_list_df = categories_df.withColumn("category", explode(categories_df["categories_list"]))

# Joindre avec le DataFrame Business pour obtenir les ID_Categorie
categories_list_df = categories_list_df.join(Business, on="business_id")

# Sélectionner les colonnes finales et renommer
Categories_List = categories_list_df.select(col("ID_Categorie"), col("category").alias("Name"))

# Afficher le DataFrame final
Categories_List.show()


25/02/20 16:08:39 WARN TaskSetManager: Stage 6 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.
25/02/20 16:08:41 WARN TaskSetManager: Stage 7 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+------------+--------------------+
|ID_Categorie|                Name|
+------------+--------------------+
|          12|            Desserts|
|          12|                Food|
|          12|Ice Cream & Froze...|
|         219|            Shopping|
|         219|     Decks & Railing|
|         219|Paint-Your-Own Po...|
|         219|       Arts & Crafts|
|         219|           Cabinetry|
|         219|       Home Services|
|         219|          Carpenters|
|         219|         Contractors|
|         737|Garage Door Services|
|         737|       Home Services|
|         758|            Caterers|
|         758|      Specialty Food|
|         758|Event Planning & ...|
|         758|      American (New)|
|         758|         Food Trucks|
|         758|American (Traditi...|
|         758|         Restaurants|
+------------+--------------------+
only showing top 20 rows



### Attributes

In [5]:
Attributes = Business.select("ID_Attributes")
Attributes.show()

+-------------+
|ID_Attributes|
+-------------+
|            0|
|            1|
|            2|
|            3|
|            4|
|            5|
|            6|
|            7|
|            8|
|            9|
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
+-------------+
only showing top 20 rows



25/02/20 16:15:58 WARN TaskSetManager: Stage 4 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


### Attributes_List

In [6]:
#Sélectionner les colonnes nécessaires
attributes_df = business_source.select("business_id", "attributes")

# Exploser les colonnes du DataFrame attributes_df
attributes_df = attributes_df.select("business_id", explode(attributes_df["attributes"]).alias("attribute", "value"))

# Joindre avec le DataFrame Business pour obtenir les ID_Attributes
attributes_df = attributes_df.join(Business, on="business_id")

# Sélectionner les colonnes finales et renommer
Attributes_List = attributes_df.select(col("ID_Attributes"), col("attribute").alias("Name"), col("value"))

# Afficher le DataFrame final
Attributes_List.show()

25/02/20 16:16:00 WARN TaskSetManager: Stage 5 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.
25/02/20 16:16:01 WARN TaskSetManager: Stage 6 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------------+--------------------+
|ID_Attributes|                Name|               value|
+-------------+--------------------+--------------------+
|           12|         BikeParking|                True|
|           12|BusinessAcceptsCr...|                True|
|           12|              Caters|                True|
|           12|     BusinessParking|{'garage': False,...|
|           12|RestaurantsPriceR...|                   1|
|           12|WheelchairAccessible|                True|
|           12|  RestaurantsTakeOut|                True|
|           12|                WiFi|               u'no'|
|          219|   ByAppointmentOnly|                True|
|          219|BusinessAcceptsCr...|               False|
|          219|BusinessAcceptsBi...|               False|
|          219|WheelchairAccessible|               False|
|          737|   ByAppointmentOnly|               False|
|          737|BusinessAcceptsCr...|                True|
|          758

### Hours

In [18]:
Hours = Business.select("ID_Hours", "business_id")
Hours = Hours.join(business_source.select("business_id", "hours"), on="business_id")
Hours = Hours.withColumn("Monday", col("hours")["Monday"])
Hours = Hours.withColumn("Tuesday", col("hours")["Tuesday"])
Hours = Hours.withColumn("Wednesday", col("hours")["Wednesday"])
Hours = Hours.withColumn("Thursday", col("hours")["Thursday"])
Hours = Hours.withColumn("Friday", col("hours")["Friday"])
Hours = Hours.withColumn("Saturday", col("hours")["Saturday"])
Hours = Hours.withColumn("Sunday", col("hours")["Sunday"])
Hours = Hours.drop("hours")
Hours = Hours.drop("business_id")
Hours.show()

25/02/20 16:40:16 WARN TaskSetManager: Stage 32 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.
25/02/20 16:40:16 WARN TaskSetManager: Stage 33 contains a task of very large size (9780 KiB). The maximum recommended task size is 1000 KiB.


+--------+-----------+-----------+-----------+-----------+----------+----------+----------+
|ID_Hours|     Monday|    Tuesday|  Wednesday|   Thursday|    Friday|  Saturday|    Sunday|
+--------+-----------+-----------+-----------+-----------+----------+----------+----------+
|      12|    0:0-0:0|  11:0-21:0|  11:0-21:0|  11:0-21:0| 11:0-22:0| 11:0-22:0| 11:0-21:0|
|     219|   7:0-18:0|   7:0-18:0|   7:0-18:0|   7:0-18:0|  7:0-18:0|  7:0-18:0|      NULL|
|     737|   8:0-17:0|   8:0-17:0|   8:0-17:0|   8:0-17:0|  8:0-17:0|  8:0-12:0|   0:0-0:0|
|     758|    6:0-0:0|    6:0-0:0|    6:0-0:0|    6:0-0:0|   6:0-0:0|   6:0-0:0|   6:0-0:0|
|    1260|       NULL|   9:0-15:0|   9:0-17:0|   9:0-17:0|  9:0-17:0|  9:0-17:0|      NULL|
|    1273|    0:0-0:0|    0:0-0:0|    0:0-0:0|    0:0-0:0|   0:0-0:0|   0:0-0:0|   0:0-0:0|
|    1607|    0:0-0:0|    0:0-0:0|    0:0-0:0|    0:0-0:0|   0:0-0:0|   0:0-0:0|   0:0-0:0|
|    2056|  9:30-19:0|  9:30-19:0|  9:30-19:0|  9:30-19:0| 9:30-19:0|  9:0-19:0|